In [1]:
!pip install transformers torch spacy sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.8 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, MarianMTModel, MarianTokenizer
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from torch.nn import BCEWithLogitsLoss
import spacy
import random
import pandas as pd

In [3]:
class BertPreprocessor:
    def __init__(self, model_name='bert-base-uncased', max_length=256):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.max_length = max_length

    def preprocess(self, texts):
        encoding = self.tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            max_length=self.max_length,
            return_attention_mask=True,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoding['input_ids'], encoding['attention_mask']

In [4]:
class BackTranslationAugmentor:
    def __init__(self, src_lang="en", target_lang="fr"):
        self.src_lang = src_lang
        self.target_lang = target_lang
        self.src_model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{target_lang}'
        self.target_model_name = f'Helsinki-NLP/opus-mt-{target_lang}-{src_lang}'
        self.src_tokenizer = MarianTokenizer.from_pretrained(self.src_model_name)
        self.target_tokenizer = MarianTokenizer.from_pretrained(self.target_model_name)
        self.src_model = MarianMTModel.from_pretrained(self.src_model_name)
        self.target_model = MarianMTModel.from_pretrained(self.target_model_name)

    def translate(self, texts, model, tokenizer):
        # Ensure texts is a list
        if isinstance(texts, str):
            texts = [texts]
        # Tokenize the texts and handle potential long sequences
        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        translated = model.generate(**inputs)
        return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

    def back_translate(self, text):
        translated_text = self.translate(text, self.src_model, self.src_tokenizer)
        back_translated_text = self.translate(translated_text, self.target_model, self.target_tokenizer)
        return back_translated_text[0]

In [5]:
class BertModel:
    def __init__(self, model_name='bert-base-uncased'):
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def train(self, input_ids, attention_masks, labels, batch_size=16, epochs=4):
        dataset = TensorDataset(input_ids, attention_masks, labels)
        train_size = int(0.9 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        optimizer = AdamW(self.model.parameters(), lr=2e-5)
        loss_fct = BCEWithLogitsLoss()

        for epoch in range(epochs):
            self.model.train()
            total_loss = 0
            for batch in train_dataloader:
                b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
                self.model.zero_grad()

                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = outputs.logits

                b_labels = b_labels.unsqueeze(1)
                b_labels = torch.cat((1 - b_labels, b_labels), dim=1)

                loss = loss_fct(logits, b_labels.float())
                total_loss += loss.item()
                loss.backward()
                optimizer.step()

            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.2f}")

            self.model.eval()
            total_eval_accuracy = 0
            for batch in val_dataloader:
                b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
                with torch.no_grad():
                    outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
                logits = outputs.logits
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                predictions = np.argmax(logits, axis=1)
                total_eval_accuracy += accuracy_score(label_ids, predictions)

            avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
            print(f"Validation Accuracy: {avg_val_accuracy:.2f}")

    def evaluate(self, input_ids, attention_masks, labels, batch_size=16):
        test_dataset = TensorDataset(input_ids, attention_masks, labels)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        self.model.eval()
        total_eval_accuracy = 0
        predictions, true_labels = [], []

        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = [t.to(self.device) for t in batch]
            with torch.no_grad():
                outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            logits = outputs.logits
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            batch_predictions = np.argmax(logits, axis=1)
            predictions.extend(batch_predictions)
            true_labels.extend(label_ids)

        print("Accuracy:", accuracy_score(true_labels, predictions))
        print("Classification Report:")
        print(classification_report(true_labels, predictions))

In [6]:
from google.colab import files
uploaded = files.upload()

Saving train_data.tsv to train_data.tsv
Saving test_data.tsv to test_data.tsv
Saving Validation_data.tsv to Validation_data.tsv


In [7]:
def main():
    # Load the datasets
    df_train = pd.read_csv('train_data.tsv', sep='\t')
    df_test = pd.read_csv('test_data.tsv', sep='\t')
    df_validate = pd.read_csv('Validation_data.tsv', sep='\t')

    # Define the column names
    columns = ['index', 'id', 'label', 'statement', 'subject', 'speaker', 'JobTitle', 'State', 'Party', 'BTC', 'FC', 'HT', 'MT', 'POF', 'context', 'justification']
    df_train.columns = columns
    df_test.columns = columns
    df_validate.columns = columns

    # Map labels to binary values for each dataset
    label_mapping = {
        'true': 1, 'half-true': 1, 'mostly-true': 1,
        'false': 0, 'pants-fire': 0, 'barely-true': 0
    }
    df_train['label'] = df_train['label'].map(label_mapping)
    df_test['label'] = df_test['label'].map(label_mapping)
    df_validate['label'] = df_validate['label'].map(label_mapping)

    # Text augmentation for the training dataset
    augmentor = BackTranslationAugmentor()
    augmentation_indices = random.sample(range(len(df_train)), int(len(df_train) * 0.1))
    augmented_rows = []
    for idx in augmentation_indices:
        original_text = df_train.loc[idx, 'statement']
        augmented_text = augmentor.back_translate(original_text)
        augmented_rows.append({'statement': augmented_text, 'label': df_train.loc[idx, 'label']})

    df_augmented = pd.DataFrame(augmented_rows)
    df_train_augmented = pd.concat([df_train, df_augmented], ignore_index=True)

    # Preprocess text data for each dataset
    bert_preprocessor = BertPreprocessor()

    def preprocess_dataset(df):
        texts = df['statement'].tolist()
        input_ids, attention_masks = bert_preprocessor.preprocess(texts)
        y_bert = torch.tensor(df['label'].values)
        return input_ids, attention_masks, y_bert

    # Prepare datasets
    input_ids_train, attention_masks_train, y_bert_train = preprocess_dataset(df_train_augmented)
    input_ids_test, attention_masks_test, y_bert_test = preprocess_dataset(df_test)
    input_ids_validate, attention_masks_validate, y_bert_validate = preprocess_dataset(df_validate)

    # Initialize the BERT model
    bert_model = BertModel()

    # Train the BERT model with the training dataset
    bert_model.train(input_ids_train, attention_masks_train, y_bert_train)
    print("Finished Training")

    # Evaluate on the Test Dataset
    print("Beginning Evaluation on Test Dataset")
    bert_model.evaluate(input_ids_test, attention_masks_test, y_bert_test)

    # Evaluate on the Validation Dataset
    print("Beginning Evaluation on Validation Dataset")
    bert_model.evaluate(input_ids_validate, attention_masks_validate, y_bert_validate)

    print("Training")
    bert_model.evaluate(input_ids_train, attention_masks_train, y_bert_train)

main()


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/4 - Loss: 0.66
Validation Accuracy: 0.60
Epoch 2/4 - Loss: 0.58
Validation Accuracy: 0.64
Epoch 3/4 - Loss: 0.40
Validation Accuracy: 0.66
Epoch 4/4 - Loss: 0.18
Validation Accuracy: 0.67
Finished Training
Beginning Evaluation on Test Dataset
Accuracy: 0.8404423380726699
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.79      0.81       553
           1       0.84      0.88      0.86       713

    accuracy                           0.84      1266
   macro avg       0.84      0.83      0.84      1266
weighted avg       0.84      0.84      0.84      1266

Beginning Evaluation on Validation Dataset
Accuracy: 0.8332034294621979
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       615
           1       0.82      0.88      0.85       668

    accuracy                           0.83      1283
   macro avg       0.84      0.83      0.83      1283
wei